<a href="https://colab.research.google.com/github/luisanovaes/ufam-project/blob/master/Code_summarization_coliee-sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv
!pip install openai

In [3]:
import requests
import os

#Acesso ao drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import pandas as pd


def load_data(drive_folder_path):
  documents = []
  list_of_ids = []
  i=0
  for txt in glob(os.path.join(drive_folder_path, '*.txt')):
    with open(txt, encoding='utf-8') as tfile:
      text = tfile.read()
      text = text.replace('\n',' ').replace('FRAGMENT_SUPPRESSED','')
      documents.append(text)
      list_of_ids.append(txt)
      list_of_ids[:] = [d.replace(drive_folder_path,'') for d in list_of_ids]
  return documents, list_of_ids

def get_segments(document):
  # Get segments from txt by splitting on .
  segments =  document.split('.')
  # Put the . back in
  segments = [segment + '.' for segment in segments]
  # Further split by comma
  segments = [segment.split(',') for segment in segments]
  # Flatten
  segments = [item for sublist in segments for item in sublist]
  return segments


def create_sentences(segments, MIN_WORDS, MAX_WORDS):

  # Combine the non-sentences together
  sentences = []

  is_new_sentence = True
  sentence_length = 0
  sentence_num = 0
  sentence_segments = []

  for i in range(len(segments)):
    if is_new_sentence == True:
      is_new_sentence = False
    # Append the segment
    sentence_segments.append(segments[i])
    segment_words = segments[i].split(' ')
    sentence_length += len(segment_words)

    # If exceed MAX_WORDS, then stop at the end of the segment
    # Only consider it a sentence if the length is at least MIN_WORDS
    if (sentence_length >= MIN_WORDS and segments[i][-1] == '.') or sentence_length >= MAX_WORDS:
      sentence = ' '.join(sentence_segments)
      sentences.append({
        'sentence_num': sentence_num,
        'text': sentence,
        'sentence_length': sentence_length
      })
      # Reset
      is_new_sentence = True
      sentence_length = 0
      sentence_segments = []
      sentence_num += 1

  return sentences

def create_chunks(sentences, CHUNK_LENGTH, STRIDE):

  sentences_df = pd.DataFrame(sentences)

  chunks = []
  for i in range(0, len(sentences_df), (CHUNK_LENGTH - STRIDE)):
    chunk = sentences_df.iloc[i:i+CHUNK_LENGTH]
    chunk_text = ' '.join(chunk['text'].tolist())

    chunks.append({
      'start_sentence_num': chunk['sentence_num'].iloc[0],
      'end_sentence_num': chunk['sentence_num'].iloc[-1],
      'text': chunk_text,
      'num_words': len(chunk_text.split(' '))
    })

  chunks_df = pd.DataFrame(chunks)
  return chunks_df.to_dict('records')


In [58]:
def create_sentences2(segments, MIN_WORDS, MAX_WORDS):

    sentences = []

    is_new_sentence = True
    sentence_length = 0
    sentence_num = 0
    sentence_segments = []

    for i in range(len(segments)):
        if is_new_sentence:
            is_new_sentence = False
        # Append the segment
        sentence_segments.append(segments[i])
        segment_words = segments[i].split(' ')
        sentence_length += len(segment_words)

        # If exceed MAX_WORDS, then stop at the end of the segment
        # Only consider it a sentence if the length is at least MIN_WORDS
        if (sentence_length >= MIN_WORDS and segments[i] and segments[i][-1] == '.') or sentence_length >= MAX_WORDS:
            sentence = ' '.join(sentence_segments)
            sentences.append({
                'sentence_num': sentence_num,
                'text': sentence,
                'sentence_length': sentence_length
            })
            # Reset
            is_new_sentence = True
            sentence_length = 0
            sentence_segments = []
            sentence_num += 1

    return sentences


##Code to import documents

In [30]:
from glob2 import glob
# Caminho para a pasta do Google Drive montada no Colab
drive_folder_path = '/content/drive/MyDrive/COLIEE-2023/Amostra-COLIEE/'  # Substitua 'sua_pasta' pelo nome da sua pasta no Google Drive


#load data (documents and lista of ids)
documents, list_of_id = load_data(drive_folder_path)
#remove blank space from id's name
list_of_ids = [item.replace(' ', '') for item in list_of_id]


#create a dictionary document->id document
id_to_doc = dict(zip(list_of_ids,documents))
doc_to_id = dict(zip(documents,list_of_ids))

In [35]:
#mypath = '/content/drive/MyDrive/099814.txt'
#text = ""

#with open(mypath, encoding='utf-8') as tfile:
#    txt099814 = tfile.read()
#    txt099814 = txt099814.replace('\n', ' ').replace('FRAGMENT_SUPPRESSED', '')

#segments = get_segments(txt099814)

In [62]:
!pip install transformers
!pip install sentencepiece
!pip install colabtools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sentencepiece
import IPython.display as display


tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
#tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")
#model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus")

In [ ]:
id_to_chunk={}
id_to_encoded_chunk = {}
for document in documents:
  segment = get_segments(document)
  sentences = create_sentences(segment, MIN_WORDS=3, MAX_WORDS=80)
  chunks = create_chunks(sentences, CHUNK_LENGTH=19, STRIDE=1)
  chunks_text = [chunk['text'] for chunk in chunks]
  id_to_chunk[doc_to_id[document]] = chunks_text
  # Tokenize each chunk.
  encoded_chunks = []
  for chunk in chunks:
    encoded_chunk = tokenizer.encode(chunk, return_tensors='pt', max_length=1024, truncation=True)
    encoded_chunks.append(encoded_chunk)
  id_to_encoded_chunk[doc_to_id[document]] = encoded_chunks





# Stage 1: Getting Chunk Summaries

In [ ]:
#1) Read all docs from a paste, 2) break in chunks, --OK
#3) generate a summary for each chunk
#4) concatenate all summaries and save in a paste
#5) create a summary from the concatenation of all summaries and save in another paste


# Stage 2: Topification of summaries

In [ ]:
# Generate a summary for each chunk.
summaries = []
for chunk in encoded_chunks:
    summary_ids = model.generate(chunk,
                                 num_beams=9,
                                 no_repeat_ngram_size=3,
                                 length_penalty=2.0,
                                 min_length=50,
                                 max_length=250,
                                 early_stopping=True)

    # Decode each summary chunk
    summary = ' '.join(tokenizer.decode(token_id) for token_id in summary_ids.squeeze())
    summaries.append(summary)

# Concatenate the summaries of each chunk.
summary = ' '.join(summaries)

#Save summary

In [ ]:
## Generate a summary for each chunk.

summaries = []
for chunk in encoded_chunks:
    summary_ids = model.generate(chunk,
                                 num_beams=9,
                                 no_repeat_ngram_size=3,
                                 length_penalty=2.0,
                                 min_length=50,
                                 max_length=250,
                                 early_stopping=True)

    # Decode each summary chunk
    summary = ' '.join(tokenizer.decode(token_id) for token_id in summary_ids.squeeze())
    summaries.append(summary)

# Concatenate the summaries of each chunk.
summary = ' '.join(summaries)